In [1]:
import requests as r
import pandas as pd
import re,os
from datetime import datetime
import json

In [2]:
import csv,json

def deduplicate_list_of_set(l):
    seen = set()
    result = []
    keys = list(l[0].keys())
    for dic in l:
        item = []
        for key in keys:
            item.append(dic[key])
        item_tuple = tuple(item)
        if item_tuple in seen:
            continue
        result.append(dic)
        seen.add(item_tuple)
    return result
        

def get_metadata_list(game_name)->list:
    metadata_list = []
    for i in range(0,6):
        with open(f'video_list/list_{i}.json') as fin:
            js = json.load(fin)
            metadata_list += js['data']
            print(len(metadata_list)) 
    metadata_list = deduplicate_list_of_set(metadata_list) 
    return metadata_list
    
metadata_list = get_metadata_list('minecraft')


99
198
296
396
495
595


In [3]:
def handle_chat_log(metadata_list):
    import os
    valid_list = []
    with open('video_list/urls.txt', 'w') as f:
        for id, metadata in enumerate(metadata_list):
            url = metadata['url']
            f.write(url+'\n')
        # chat = ChatDownloader().get_chat(url) 
        # for message in chat:                        # iterate over messages
        #     chat.print_formatted(message)
        # with open(f"{id}.json", "w")as f:
        #     f.write(json.dumps(chat, ensure_ascii=False))
        

handle_chat_log(metadata_list)

In [12]:
with open('scrap_chat_log.sh', 'w')as f:
    with open('video_list/urls.txt') as fin:
        raw = fin.readlines()
    print(f"Total {len(raw)} urls.")
    for i, line in enumerate(raw):
        f.write(f'chat_downloader {line.strip()} --output chat_log/{i}.json\n')

Total 590 urls.


In [4]:
def filter_urls():
    l = []
    for file in os.listdir('chat_log'):
        l.append(int(file.split('.')[0]))
    with open('video_list/urls.txt') as fin:
        raw = fin.readlines()
        print(f"Before filtering, we have total {len(raw)} urls.")
    # filtered_l = []
    # for i in l:
    #     filtered_l.append(raw[i])
    # with open('video_list/filtered_urls.txt', 'w') as fout:
    #     for line in filtered_l:
    #         fout.write(line)
    # print(f"After filtering, we have total {len(filtered_l)} urls.")

In [3]:
from spacy_langdetect import LanguageDetector
import spacy
from langdetect import detect

def is_english(sent):
    result = detect("Hello world!")
    if result == "en":
        return True
    else:
        return False

print(("Hello world!"))

/Users/caohanwen/miniconda3/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hello world!


In [5]:
def filter_metadata_list():
    filtered_list = []
    urls = []
    with open("video_list/filtered_urls.txt") as fin:
        raw = fin.readlines()
    for url in raw:
        urls.append(url.strip())
    print(f'before filtering, we have {len(metadata_list)} videos.')
    for meta in metadata_list:
        if meta['url'] in urls:
            filtered_list.append(meta)
    print(f'should have {len(urls)} videos.')
    print(f'filtered out {len(filtered_list)} videos.')
    return filtered_list


metadata_list = filter_metadata_list()

before filtering, we have 590 videos.
should have 567 videos.
filtered out 567 videos.


In [ ]:
def rearrange_chat_log_filename():
    import os
    filelist = os.listdir("../chat_log")
    original_idx = []
    for name in filelist:
        original_idx.append(int(name.split('.')[0]))
    original_idx = sorted(original_idx)
    for i, original in enumerate(original_idx):
        os.rename(f"../chat_log/{original}.json", f"../chat_log/{i}.json")


In [7]:
import json,csv
from spacy_langdetect import LanguageDetector
import spacy

def write_to_csv(metadata_list):
    from datetime import datetime,timezone,timedelta

    def get_time(start_time):
        start_time_obj = datetime.fromisoformat(start_time[:-1]).astimezone(timezone.utc)
        return str(start_time_obj) 

    def add_time_eps(start_time, time_in_seconds:int):
        start_time_obj = datetime.fromisoformat(start_time[:-1]).astimezone(timezone.utc)
        delta = timedelta(seconds = time_in_seconds)
        return str(delta + start_time_obj)

    # gift price
    price = {1: 4.99, 2: 9.99, 3: 24.99,}
    fieldnames = ['id', # identifier of the video 
        'stream_id',
        'user_id',
        'user_login',
        'user_name',
        'title',
        'description',
        'url',
        'thumbnail_url',
        'viewable',
        'view_count',
        'language',
        'type',
        'duration',
        'video_created_at',
        'comment_created_at',
        'muted_segments',
        'time_in_seconds',  # time of comment sent after video begins (in second)
        'comment',  # comment message
        'author_name', # comment user
        'author_id', # identifier of author
        'display_author_name', # The name of the author which is displayed to the viewer. This may be different to name.
        'author_type',
        'author_images',
        'author_badges',
        'author_subscribe_status',
        'author_gender',
        'author_is_banned',
        'author_is_bot',
        'author_is_non_coworker',
        'author_is_original_poster',
        'author_is_verified',
        'message_type', # type of message
        'emotes',
        # gift states
        'has_gift',
        'gift_type',
        'gift_tier',
        'gift_amount',
        'gift_sum',
        'gift_receiver',
        ]
    filename_list = os.listdir('chat_log')
    for step, filename in enumerate(filename_list):
        # record information about last community-gift-giving deed
        try:
            i = int(filename.split('.')[0])
        except:
            print(f"Invalid filename {filename}. Skip")
            continue
        last_sender = None
        with open(f"clean/{i}.csv", 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames = fieldnames)   
            writer.writeheader() 
            with open(f'chat_log/{filename}') as fin:
                js = json.load(fin)
            for item in js:
                row = {'id': i, }
                try:
                    del metadata_list[i]['id']
                except KeyError:
                    pass
                try:
                    del metadata_list[i]['language']# all english!
                except KeyError:
                    pass
                published_at = None
                published_at_str = None
                created_at = metadata_list[i]['created_at']
                # del metadata_list[i]['created_at'] 
                published_at = metadata_list[i]['published_at']
                # del metadata_list[i]['published_at']
                assert(created_at == published_at)
                published_at_str = get_time(published_at) 
                row['video_created_at'] = published_at_str
                row.update(metadata_list[i])
                try:
                    row['time_in_seconds'] = item['time_in_seconds']
                    row['comment_created_at'] = add_time_eps(published_at, item['time_in_seconds'])
                except KeyError:
                    pass
                try:
                    row['comment'] =  item['message']
                    give_all_substr = 'all the shineys'
                    give_individual_substr = 'gifted a Tier' 
                    give_community_substr = 'is gifting'
                    if give_all_substr in item['message']:
                        row['has_gift'] = True
                        row['gift_type'] = "Give all"
                    elif give_community_substr in item['message']:
                        try:
                            last_sender = item['author']['id']
                            msg = item['message'][(item['message'].find(give_community_substr) + len(give_community_substr)):] 
                            msg = msg.split()
                            #  5 Tier 1 Subs to Ninja's community!
                            row['has_gift'] = True
                            row['gift_tier'] = int(msg[2])
                            row['gift_amount'] = int(msg[0])
                            row['gift_receiver'] = msg[5] + " " + msg[6]
                            row['gift_sum'] = row['gift_amount'] * price[row['gift_tier']]
                            row['gift_type'] = "To community" 
                        except :
                            print("?")
                            print(msg)
                            row['has_gift'] = False  
                    elif give_individual_substr in item['message']:
                        if last_sender != item['author']['id']:
                            try:
                                msg = item['message'][item['message'].find(give_individual_substr) + len(give_individual_substr):]
                                msg = msg.split()
                                row['has_gift'] = True
                                row['gift_tier'] = int(msg[0])
                                row['gift_amount'] = 1
                                row['gift_receiver'] = msg[3]
                                row['gift_sum'] = row['gift_amount'] * price[row['gift_tier']]
                                row['gift_type'] = "To specific user"
                            except:
                                print("??")
                                print(msg)
                                row['has_gift'] = False  
                        else:
                            row['has_gift'] = False 
                    else:
                        row['has_gift'] = False
                except KeyError:
                    pass
                try:
                    row['author_name'] = item['author']['name']
                except KeyError:
                    pass
                try:
                    row['author_id'] = item['author']['id']
                except KeyError:
                    pass
                try:
                    row['display_author_name'] = item['author']['display_name']
                except KeyError:
                    pass
                try:
                    row['author_type'] = item['author']['type']
                except KeyError:
                    pass
                try:
                    row['author_images'] = item['author']['images']
                except KeyError:
                    pass
                try:
                    row['author_badges'] = item['author']['badges']
                    for badge in item['author']['badges']:
                        try:
                            row['author_subscribe_status'] +=  (badge['description'] + ",")
                        except KeyError:
                            pass
                except KeyError:
                    pass
                try:
                    row['author_gender'] = item['author']['gender']
                except KeyError:
                    pass
                try:
                    row['author_is_banned'] = item['author']['is_banned']
                except:
                    pass
                try:
                    row['author_is_bot' ] = item['author']['is_bot']
                except KeyError:
                    pass
                try:
                    row['author_is_non_coworker'] =  item['author']['is_non_coworker']
                except KeyError:
                    pass
                try:
                    row['author_is_original_poster'] = item['author']['is_original_poster']
                except KeyError:
                    pass
                try:
                    row['author_is_verified'] =item['author']['is_verified']
                except KeyError:
                    pass
                try:
                    row['message_type'] = item['message_type']
                except KeyError:
                    pass
                try:
                    row['emotes'] = item['emotes']
                except KeyError:
                    pass
                try:
                    del row["created_at"]
                    del row["published_at"]
                except:
                    pass
                writer.writerow(row)
            
write_to_csv(metadata_list)

?
['the', '15?']
?
['22', 'Tier', '1', 'Subs', 'to', "HannahSayce's", 'community!', "They've", 'gifted', 'a', 'total', 'of', '155', 'in', 'the', 'channel!']
?
['1', 'sub', 'to', 'mafusato!', 'They', 'have', 'given', '4', 'Gift', 'Subs', 'in', 'the', 'channel!']
Invalid filename .DS_Store. Skip
?
['subs']
?
['5', 'Tier', '1', 'Subs', 'to', "jacksepticeye's", 'community!', "They've", 'gifted', 'a', 'total', 'of', '9', 'in', 'the', 'channel!']
?
['1', 'sub', 'to', 'nj_grifondoro!', 'This', 'is', 'their', 'first', 'Gift', 'Sub', 'in', 'the', 'channel!']
?
['5', 'Tier', '1', 'Subs', 'to', "Ninja's", 'community!', "They've", 'gifted', 'a', 'total', 'of', '11', 'in', 'the', 'channel!']
?
['1', 'Tier', '1', 'Subs', 'to', "Ninja's", 'community!', "They've", 'gifted', 'a', 'total', 'of', '2', 'in', 'the', 'channel!']
?
['1', 'Tier', '1', 'Subs', 'to', "Ninja's", 'community!', "They've", 'gifted', 'a', 'total', 'of', '12', 'in', 'the', 'channel!']
?
['25', 'Tier', '1', 'Subs', 'to', "Ninja's", 'c